In [1]:
import sys
import os
sys.path.append(os.getenv("PYTHONPATH", "/app")) #REVIEW
from utils import Utils

In [2]:
utils = Utils()
spark = utils.get_spark_session()
emp = utils.get_employee_data()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/08 12:25:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
# Window functions in pyspark can be defined through the Window class and then the .over() method can be called on a ranking function to apply it over this window:
# Getting the max salary for every department
from pyspark.sql.window import Window
from pyspark.sql.functions import col, max

window = Window.partitionBy(col("department_id"))

emp.withColumn("max_department_salary", max("salary").over(window)).orderBy(col("department_id")).show()

In [ ]:
# Getting the second biggest salary for every department
from pyspark.sql.functions import row_number

window = Window.partitionBy(col("department_id")).orderBy(col("salary").desc()) # In this case, this is a ranking function and not and aggregation function like max() so data has to be ordered

emp.withColumn("department_salary_rank", row_number().over(window)).orderBy(col("department_id")).where("department_salary_rank == 2").show()

In [ ]:
# The same table can be generated with SQL synthax via the expr() function:
from pyspark.sql.functions import expr

emp.withColumn("department_salary_rank", expr("row_number() over(partition by department_id order by salary desc)")).where("department_salary_rank == 2").show()

In [ ]:
# To rank a metric withouth taking partitions into accout:

window = Window.orderBy(col("salary").desc())

emp.withColumn("salary_rank", row_number().over(window)).show()

In [ ]:
# This is an example of other rank functions over the same window:
from pyspark.sql.functions import row_number, rank, ntile, dense_rank

emp.withColumn("row_number", row_number().over(window)) \
  .withColumn("rank", rank().over(window)) \
  .withColumn("dense_rank", dense_rank().over(window)) \
  .withColumn("ntile", ntile(2).over(window)) \
  .show()